# Circuit Composition - Complete Guide for Qiskit Certification

**Section 3: Create Circuits - Part 2**

> **Exam Weight**: Part of 18% (HIGHEST WEIGHT!) | **Must Master**: ✅✅✅

This notebook covers methods for combining multiple quantum circuits into larger, more complex circuits - essential for building modular quantum algorithms!

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- Master `compose()` for sequential circuit combination
- Understand `tensor()` for parallel composition
- Use `append()` to add gates and subcircuits
- Identify exam traps with compose vs tensor
- Build complex circuits from simple components

---

## 💡 Conceptual Deep Dive: The LEGO Analogy

**Circuit Composition = Building with LEGO**

| LEGO Building | Quantum Circuits |
|---------------|------------------|
| **Single brick** | Individual gate |
| **Pre-built module** | Sub-circuit |
| **Stacking vertically** | `compose()` - sequential |
| **Building side-by-side** | `tensor()` - parallel |
| **Attaching one brick** | `append()` - add operation |

```
compose():  [A]→[B]     # A then B (same qubits, time direction)
tensor():   [A]⊗[B]     # A and B separate (more qubits, space direction)
append():   [Circuit]+[gate] # Add one operation to end
```

**Key Insight**:
- `compose()` = time evolution (same qubits, more depth)
- `tensor()` = space expansion (more qubits, independent systems)
- `append()` = single step addition

---

## 🔬 Critical Exam Concept: compose() vs tensor()

**This is the #1 tested concept in Section 3!**

```python
# compose(): Sequential - SAME qubits, operations happen AFTER
result = qc1.compose(qc2)  # qc1 gates → then → qc2 gates
# Width stays the same!

# tensor(): Parallel - DIFFERENT qubits, systems are INDEPENDENT  
result = qc1.tensor(qc2)   # qc1 qubits ⊗ qc2 qubits
# Width INCREASES!
```

| Feature | compose() | tensor() |
|---------|-----------|----------|
| **Direction** | Time (→) | Space (⊗) |
| **Qubits** | Same | Combined (new) |
| **Width** | Unchanged | Increased |
| **Depth** | Increased | Max of both |

---

## Setup

Import required libraries:

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFTGate, XGate, CXGate

print("✅ Imports successful!")

## Setup

Import required libraries:

## Part 1: compose() - Sequential Composition (EXAM CRITICAL!)

In [ ]:
# Basic compose - sequential execution
qc1 = QuantumCircuit(2)
qc1.h([0, 1])

qc2 = QuantumCircuit(2)
qc2.cx(0, 1)

print("Circuit 1 (preparation):")
print(qc1.draw())
print("\nCircuit 2 (entanglement):")
print(qc2.draw())

# Compose: qc1 THEN qc2
full_circuit = qc1.compose(qc2)
print("\nqc1.compose(qc2) - Sequential:")
print(full_circuit.draw())
print("\n✅ Gates from qc1, followed by gates from qc2")

In [ ]:
# compose with qubit mapping
qc_large = QuantumCircuit(4)
qc_large.h(range(4))

qc_small = QuantumCircuit(2)
qc_small.cx(0, 1)

# Apply small circuit to specific qubits
# result = qc_large.compose(qc_small, qubits=[1, 2])
result = qc_large.compose(qc_small, qubits=[1,2])

print("Large circuit (4 qubits):")
print(qc_large.draw())
print("\nSmall circuit (2 qubits):")
print(qc_small.draw())
print("\ncompose(qc_small, qubits=[1,2]):")
print(result.draw())
print("\n✅ Small circuit applied to qubits 1 and 2 of large circuit")

### ⚠️ EXAM TRAP: compose() vs tensor()

**Most common Section 3 exam question!**

```python
# compose() - SEQUENTIAL (same qubits, after)
qc1.compose(qc2)  # qc1 gates → qc2 gates (time)

# tensor() - PARALLEL (different qubits, side-by-side)
qc1.tensor(qc2)   # qc1 qubits ⊗ qc2 qubits (space)
```

## Part 2: tensor() - Parallel Composition

In [ ]:
# tensor - parallel systems
system1 = QuantumCircuit(2)
system1.h(0)
system1.cx(0, 1)

system2 = QuantumCircuit(2)
system2.x([0, 1])

print("System 1 (2 qubits):")
print(system1.draw())
print("\nSystem 2 (2 qubits):")
print(system2.draw())

# Tensor product: side-by-side
combined = system1.tensor(system2)
print("\nsystem1.tensor(system2) - Parallel:")
print(combined.draw())
print(f"\n✅ Total qubits: {combined.num_qubits} (2 + 2)")
print("   Systems operate independently (tensor product ⊗)")

### Visual Comparison: compose() vs tensor()

In [ ]:
# Create identical sub-circuits
sub = QuantumCircuit(2)
sub.h(0)
sub.cx(0, 1)

# Method 1: compose (sequential)
qc_compose = QuantumCircuit(2)
qc_compose.compose(sub, inplace=True)
qc_compose.compose(sub, inplace=True)  # Run twice

print("compose() - Sequential (same qubits):")
print(qc_compose.draw())
print(f"Qubits: {qc_compose.num_qubits}, Depth: {qc_compose.depth()}")

print("\n" + "="*50 + "\n")

# Method 2: tensor (parallel)
sub1 = QuantumCircuit(2)
sub1.h(0)
sub1.cx(0, 1)

sub2 = QuantumCircuit(2)
sub2.h(0)
sub2.cx(0, 1)

qc_tensor = sub1.tensor(sub2)

print("tensor() - Parallel (different qubits):")
print(qc_tensor.draw())
print(f"Qubits: {qc_tensor.num_qubits}, Depth: {qc_tensor.depth()}")

print("\n🎯 KEY DIFFERENCE:")
print("   compose: 2 qubits, deeper circuit (sequential)")
print("   tensor:  4 qubits, shallower circuit (parallel)")

## Part 3: append() - Adding Gates and Circuits

In [ ]:
# append gates
qc = QuantumCircuit(3)
qc.h([0, 1, 2])

# Append QFT as a gate
qc.append(QFTGate(3), [0, 1, 2])

print("Circuit with appended QFT:")
print(qc.draw())
print("\n✅ QFT appears as single block (not decomposed)")

In [ ]:
# append with custom gate
qc = QuantumCircuit(2)
qc.h(0)

# Create custom operation
bell_gate = QuantumCircuit(2, name='Bell')
bell_gate.h(0)
bell_gate.cx(0, 1)

# Append as gate
qc.append(bell_gate.to_gate(), [0, 1])

print("Circuit with custom Bell gate:")
print(qc.draw())
print("\n✅ Custom operations shown as named blocks")

## Part 4: inverse() and power() Operations

In [ ]:
# inverse() - reverse and adjoint
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.t(0)

print("Original circuit:")
print(qc.draw())

# Get inverse
# qc_inv = qc.inverse()
qc_inv = qc.inverse()

print("\nInverse circuit:")
print(qc_inv.draw())
print("\n✅ Gates reversed and inverted (T† instead of T)")

In [ ]:
# Verify inverse cancels original
qc_full = qc.compose(qc.inverse())

print("Original + Inverse:")
print(qc_full.draw())
print("\n✅ Should return to |00⟩ (identity operation)")

In [ ]:
# power() - repeat circuit
qc = QuantumCircuit(1)
qc.s(0)  # S gate (√Z)

print("Original (S gate):")
print(qc.draw())

# S² = Z
qc_squared = qc.power(2)
print("\npower(2) - S²:")
print(qc_squared.draw())
print("\n✅ S² = Z (exam fact!)")

## Part 5: decompose() - View Basic Gates

In [ ]:
# decompose complex gates
qc = QuantumCircuit(3)
qc.append(QFTGate(3), range(3))

print("Before decompose:")
print(qc.draw())

# Decompose to basic gates
qc_decomposed = qc.decompose()

print("\nAfter decompose():")
print(qc_decomposed.draw())
print("\n✅ QFT expanded into H, CRZ, SWAP gates")

## 📝 Practice Questions

### Question 1: compose() vs tensor()

**You have two 2-qubit circuits. Which creates a 4-qubit circuit?**

A) `qc1.compose(qc2)`  
B) `qc1.tensor(qc2)`  
C) `qc1.append(qc2)`  
D) `qc1.inverse()`

<details>
<summary>Answer</summary>

**B) `qc1.tensor(qc2)`**

- `compose()` - sequential on SAME qubits (2 qubits)
- `tensor()` - parallel on DIFFERENT qubits (4 qubits) ✅
- `append()` - adds gates to existing circuit
- `inverse()` - reverses circuit (same qubits)
</details>

---

### Question 2: What does inverse() do?

**If qc applies H, then T, what does qc.inverse() apply?**

A) H, then T  
B) T†, then H  
C) H, then T†  
D) T, then H

<details>
<summary>Answer</summary>

**B) T†, then H**

inverse() **reverses order** and **takes adjoint**:
- Original: H → T
- Inverse: T† → H (reverse order + adjoint)

Note: H is self-adjoint (H† = H), but T† ≠ T
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **compose()** - Sequential combination (time)
   - Same qubits, gates run one after another
   - Use: Building algorithm phases

2. **tensor()** - Parallel combination (space)  
   - Different qubits, creates larger system
   - Use: Combining independent registers

3. **append()** - Add gates or circuits
   - Flexible gate addition
   - Can append library gates or custom circuits

4. **inverse()** - Reverse and adjoint
   - Reverses gate order + takes adjoint
   - Use: Uncomputing, QFT†

5. **decompose()** - Expand to basic gates
   - Shows implementation details
   - Useful for understanding depth

### Critical Exam Facts

- ✅ **compose()** = sequential, **tensor()** = parallel (MOST TESTED!)
- ✅ compose() keeps qubit count, tensor() adds qubits
- ✅ inverse() reverses order AND takes adjoint
- ✅ decompose() expands composite gates
- ✅ append() adds single gates, compose() adds full circuits

### Mnemonic

🧠 **"Compose = Continue, Tensor = Together"**

**Next**: Parameterized Circuits for VQE and QAOA!